In [1]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image

c:\Users\hugoc\anaconda3\envs\thefreeai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
image_path = "../static/images/__899969187.png"
image = Image.open(image_path)

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected: {model.config.id2label[label.item()]}"
    )


Detected: sports ball
Detected: cat
Detected: sports ball
Detected: sports ball
Detected: sports ball
Detected: dog
Detected: dog
Detected: sports ball


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", torch_dtype=torch.bfloat16, device_map="auto")

prompt = "Generate only a list of ten relevant and popular Instagram hashtags for the following words: dog, cat, fish.  Provide only the list of hashtags, with no additional text or explanation."

# Mostrar el prompt
print("Prompt Text:")
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=1000)
output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

# Mostrar la salida de manera clara
print("Generated Text:")
print(output)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

torch.cuda.empty_cache()

model_id = "nvidia/Llama3-ChatQA-1.5-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to('cuda')

messages = [
    {"role": "user", "content": "Generate a list of ten relevant and popular Instagram hashtags for the following words: football, sport and player.  Provide only the list of hashtags, with no additional text or explanation. Give only the best 10 hashtags."}
]

document = """You are an advanced AI model specialized in social media content creation. Your task is to generate relevant and popular hashtags for Instagram posts based on a given keyword. These hashtags should help increase the visibility and engagement of the post. Consider trending topics, popular culture, and commonly used hashtags within the Instagram community. Ensure the hashtags are appropriate and related to the keyword provided."""

def get_formatted_input(messages, context):
    system = "System: This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context."
    instruction = "Please give a full and complete answer for the question."

    for item in messages:
        if item['role'] == "user":
            item['content'] = instruction + " " + item['content']
            break

    conversation = '\n\n'.join(["User: " + item["content"] if item["role"] == "user" else "Assistant: " + item["content"] for item in messages]) + "\n\nAssistant:"
    formatted_input = system + "\n\n" + context + "\n\n" + conversation
    
    return formatted_input

formatted_input = get_formatted_input(messages, document)
tokenized_prompt = tokenizer(tokenizer.bos_token + formatted_input, return_tensors="pt").to('cuda')

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(input_ids=tokenized_prompt.input_ids, attention_mask=tokenized_prompt.attention_mask, max_new_tokens=64, eos_token_id=terminators)

response = outputs[0][tokenized_prompt.input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


c:\Users\hugoc\anaconda3\envs\thefreeai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hugoc\anaconda3\envs\thefreeai\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.15s/it]
c:\Users\hugoc\anaconda3\envs\thefreeai\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a 

tensor([[128000,   2374,     25,   1115,    374,    264,   6369,   1990,    264,
           1217,    323,    459,  21075,  11478,  18328,     13,    578,  18328,
           6835,  11190,     11,  11944,     11,    323,  48887,  11503,    311,
            279,   1217,    596,   4860,   3196,    389,    279,   2317,     13,
            578,  18328,   1288,   1101,  13519,    994,    279,   4320,   4250,
            387,   1766,    304,    279,   2317,    382,   2675,    527,    459,
          11084,  15592,   1646,  28175,    304,   3674,   3772,   2262,   9886,
             13,   4718,   3465,    374,    311,   7068,   9959,    323,   5526,
          82961,    369,  14318,   8158,   3196,    389,    264,   2728,  16570,
             13,   4314,  82961,   1288,   1520,   5376,    279,  24035,    323,
          20392,    315,    279,   1772,     13,  21829,  51950,  13650,     11,
           5526,   7829,     11,    323,  17037,   1511,  82961,   2949,    279,
          14318,   4029,    